# 🔗 Causal Relationship Classifier (CrossEncoder)

**Purpose**: Detect causal relationships between ITSM tickets
- Input: Two ticket texts
- Output: Probability that Ticket A **caused** Ticket B

**Two-Stage Pipeline:**
1. **Stage 1** (your similarity model): Find top-K similar tickets
2. **Stage 2** (this model): Determine if relationship is causal

**Why CrossEncoder?**
- Designed for pair classification (NLI-style)
- Much better at subtle relationships than embedding similarity
- Separate from similarity model (no interference)

**Expected Performance:** F1 ~0.65-0.80 for causal detection

In [ ]:
# --- Environment Setup ---
import os
import sys
import subprocess

# Suppress warnings
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['WANDB_DISABLED'] = 'true'

# Install required packages
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 
                       'sentence-transformers', 'torch', 'scikit-learn', 'tqdm'])

print("✅ Environment ready")

In [ ]:
# --- Configuration ---
import random
import numpy as np
import torch

CONFIG = {
    'relationship_data': 'data/relationship_pairs.json',
    'output_dir': 'models/causal_classifier',
    'base_model': 'cross-encoder/ms-marco-MiniLM-L-6-v2',  # Good balance of speed/accuracy
    'epochs': 5,
    'batch_size': 16,
    'warmup_ratio': 0.1,
    'seed': 42,
    'max_pairs': 10000,  # Limit to prevent memory issues
}

# Set seeds
random.seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])

# Device detection
if torch.cuda.is_available():
    device = 'cuda'
    print(f"🚀 CUDA: {torch.cuda.get_device_name(0)}")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps'
    print("🍎 MPS (Apple Silicon)")
else:
    device = 'cpu'
    print("⚠️ CPU only")

print(f"📁 Output: {CONFIG['output_dir']}")

In [ ]:
# --- Load and Prepare Data ---
import json
from pathlib import Path

def resolve_data_path(path_str):
    """Smart path resolver for Local/Kaggle/Colab."""
    p = Path(path_str)
    if p.exists():
        return p.resolve()
    script_dir = Path.cwd()
    if (script_dir / path_str).exists():
        return (script_dir / path_str).resolve()
    raise FileNotFoundError(f"Could not find {path_str}")

# Load relationship data
rel_data_path = resolve_data_path(CONFIG['relationship_data'])
print(f"📂 Loading: {rel_data_path}")

with open(rel_data_path, 'r') as f:
    rel_data = json.load(f)

print(f"📊 Total pairs: {len(rel_data)}")

# Count by label
from collections import Counter
label_counts = Counter(item['label'] for item in rel_data)
print(f"📈 Label distribution:")
for label, count in sorted(label_counts.items()):
    print(f"   {label}: {count}")

In [ ]:
# --- Create Binary Causal Dataset ---
# Convert to binary: causal (1) vs non-causal (0)

causal_pairs = []
non_causal_pairs = []

for item in rel_data:
    pair_data = {
        'text_a': item['text_a'],
        'text_b': item['text_b'],
        'label': item['label']
    }
    
    if item['label'] == 'causal':
        causal_pairs.append(pair_data)
    elif item['label'] in ['none', 'related', 'duplicate']:
        non_causal_pairs.append(pair_data)

print(f"✅ Causal pairs: {len(causal_pairs)}")
print(f"✅ Non-causal pairs: {len(non_causal_pairs)}")

# Balance dataset (undersample majority class)
# Use 1:3 ratio (causal:non-causal) for better learning
max_non_causal = min(len(non_causal_pairs), len(causal_pairs) * 3)
random.shuffle(non_causal_pairs)
non_causal_sample = non_causal_pairs[:max_non_causal]

print(f"📊 After balancing: {len(causal_pairs)} causal, {len(non_causal_sample)} non-causal")

# Combine and shuffle
all_pairs = causal_pairs + non_causal_sample
random.shuffle(all_pairs)

# Limit total pairs if needed (memory safety)
if len(all_pairs) > CONFIG['max_pairs']:
    all_pairs = all_pairs[:CONFIG['max_pairs']]
    print(f"⚠️ Limited to {CONFIG['max_pairs']} pairs")

print(f"🎯 Final dataset: {len(all_pairs)} pairs")

In [ ]:
# --- Prepare Training Data for CrossEncoder ---
from sentence_transformers import InputExample
from sklearn.model_selection import train_test_split

# Create InputExamples for CrossEncoder
examples = []
for pair in all_pairs:
    label = 1.0 if pair['label'] == 'causal' else 0.0
    examples.append(InputExample(
        texts=[pair['text_a'], pair['text_b']],
        label=label
    ))

# Split into train/eval
train_examples, eval_examples = train_test_split(
    examples,
    test_size=0.2,
    random_state=CONFIG['seed'],
    stratify=[ex.label for ex in examples]
)

print(f"📊 Train: {len(train_examples)} pairs")
print(f"📊 Eval: {len(eval_examples)} pairs")

# Check label balance in splits
train_pos = sum(1 for ex in train_examples if ex.label == 1.0)
eval_pos = sum(1 for ex in eval_examples if ex.label == 1.0)
print(f"   Train: {train_pos} causal, {len(train_examples) - train_pos} non-causal")
print(f"   Eval: {eval_pos} causal, {len(eval_examples) - eval_pos} non-causal")

In [ ]:
# --- Initialize CrossEncoder ---
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from torch.utils.data import DataLoader
import gc

# Clear memory before loading model
gc.collect()
if device == 'mps':
    torch.mps.empty_cache()
elif device == 'cuda':
    torch.cuda.empty_cache()

print(f"🔄 Loading CrossEncoder: {CONFIG['base_model']}")

# Initialize model
# Note: CrossEncoder uses its own device management
model = CrossEncoder(
    CONFIG['base_model'],
    num_labels=1,  # Binary classification (sigmoid output)
    max_length=512,
    device=device
)

print(f"✅ Model loaded on {device}")
print(f"   Max sequence length: 512")

In [ ]:
# --- Setup Evaluator ---
# Prepare eval data for CEBinaryClassificationEvaluator
eval_sentence_pairs = [(ex.texts[0], ex.texts[1]) for ex in eval_examples]
eval_labels = [int(ex.label) for ex in eval_examples]

evaluator = CEBinaryClassificationEvaluator(
    sentence_pairs=eval_sentence_pairs,
    labels=eval_labels,
    name='causal_eval'
)

print(f"✅ Evaluator ready with {len(eval_labels)} pairs")

In [ ]:
# --- Train CrossEncoder ---
from datetime import datetime

# Create output directory
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
save_path = Path(CONFIG['output_dir']) / f"causal_{timestamp}"
save_path.mkdir(parents=True, exist_ok=True)

# Prepare DataLoader
train_dataloader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=CONFIG['batch_size']
)

# Calculate warmup steps
total_steps = len(train_dataloader) * CONFIG['epochs']
warmup_steps = int(total_steps * CONFIG['warmup_ratio'])

print(f"🚀 Starting training...")
print(f"   Epochs: {CONFIG['epochs']}")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Total steps: {total_steps}")
print(f"   Warmup steps: {warmup_steps}")
print(f"   Output: {save_path}")

# Train
model.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,
    epochs=CONFIG['epochs'],
    warmup_steps=warmup_steps,
    output_path=str(save_path),
    save_best_model=True,
    show_progress_bar=True
)

print("✅ Training complete!")

In [ ]:
# --- Evaluate Final Model ---
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns

# Load best model
best_model = CrossEncoder(str(save_path), device=device)

# Predict on eval set
predictions = best_model.predict(eval_sentence_pairs)

# Convert to binary predictions (threshold = 0.5)
binary_preds = (predictions > 0.5).astype(int)

# Metrics
print("="*60)
print("📊 FINAL EVALUATION RESULTS")
print("="*60)

print("\n📋 Classification Report:")
print(classification_report(eval_labels, binary_preds, target_names=['Non-Causal', 'Causal']))

# ROC-AUC
roc_auc = roc_auc_score(eval_labels, predictions)
print(f"🎯 ROC-AUC: {roc_auc:.4f}")

# Find optimal threshold using PR curve
precision, recall, thresholds = precision_recall_curve(eval_labels, predictions)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
best_f1 = f1_scores[best_idx]

print(f"🎯 Best F1: {best_f1:.4f} @ threshold={best_threshold:.3f}")

# Confusion matrix
cm = confusion_matrix(eval_labels, binary_preds)
print(f"\n📊 Confusion Matrix:")
print(cm)

In [ ]:
# --- Visualization ---
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. Confusion Matrix Heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Non-Causal', 'Causal'],
            yticklabels=['Non-Causal', 'Causal'])
axes[0].set_title('Confusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# 2. Score Distribution
causal_scores = [predictions[i] for i in range(len(predictions)) if eval_labels[i] == 1]
non_causal_scores = [predictions[i] for i in range(len(predictions)) if eval_labels[i] == 0]

axes[1].hist(non_causal_scores, bins=30, alpha=0.7, label='Non-Causal', color='blue')
axes[1].hist(causal_scores, bins=30, alpha=0.7, label='Causal', color='red')
axes[1].axvline(x=0.5, color='black', linestyle='--', label='Threshold (0.5)')
axes[1].axvline(x=best_threshold, color='green', linestyle='--', label=f'Best ({best_threshold:.2f})')
axes[1].set_title('Score Distribution')
axes[1].set_xlabel('Prediction Score')
axes[1].set_ylabel('Count')
axes[1].legend()

# 3. Precision-Recall Curve
axes[2].plot(recall, precision, 'b-', linewidth=2)
axes[2].scatter([recall[best_idx]], [precision[best_idx]], color='red', s=100, 
                label=f'Best F1={best_f1:.3f}', zorder=5)
axes[2].set_title('Precision-Recall Curve')
axes[2].set_xlabel('Recall')
axes[2].set_ylabel('Precision')
axes[2].legend()

plt.tight_layout()
plt.savefig(save_path / 'evaluation_plots.png', dpi=150)
plt.show()

print(f"\n✅ Plots saved to: {save_path / 'evaluation_plots.png'}")

In [ ]:
# --- Save Metadata ---
import json

metadata = {
    'training_date': datetime.now().isoformat(),
    'base_model': CONFIG['base_model'],
    'task': 'binary_causal_classification',
    'config': CONFIG,
    'device': device,
    'metrics': {
        'roc_auc': float(roc_auc),
        'best_f1': float(best_f1),
        'best_threshold': float(best_threshold),
    },
    'data_stats': {
        'total_pairs': len(all_pairs),
        'train_pairs': len(train_examples),
        'eval_pairs': len(eval_examples),
        'causal_pairs': len(causal_pairs),
        'non_causal_pairs': len(non_causal_sample),
    }
}

metadata_path = save_path / 'training_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Metadata saved to: {metadata_path}")

# 🚀 Usage: How to Use This Model

After training, use the causal classifier in your inference pipeline:

In [ ]:
# --- Example Usage ---
# This shows how to use the trained causal classifier in your inference pipeline

def detect_causal_relationship(ticket_a: str, ticket_b: str, threshold: float = None) -> dict:
    """
    Detect if ticket_a caused ticket_b.
    
    Args:
        ticket_a: Text of the potential cause ticket
        ticket_b: Text of the potential effect ticket
        threshold: Classification threshold (default: best_threshold from training)
    
    Returns:
        dict with 'is_causal' (bool), 'confidence' (float), 'threshold' (float)
    """
    if threshold is None:
        threshold = best_threshold
    
    # Get prediction score
    score = best_model.predict([(ticket_a, ticket_b)])[0]
    
    return {
        'is_causal': bool(score > threshold),
        'confidence': float(score),
        'threshold': threshold
    }

# Test examples
print("="*60)
print("🧪 EXAMPLE PREDICTIONS")
print("="*60)

test_pairs = [
    ("Server database crashed due to disk full", 
     "Multiple users reporting application timeout errors"),
    
    ("User requested password reset", 
     "Network switch firmware upgrade completed"),
    
    ("DNS server configuration changed", 
     "Website not loading for external users"),
]

for ticket_a, ticket_b in test_pairs:
    result = detect_causal_relationship(ticket_a, ticket_b)
    status = "✅ CAUSAL" if result['is_causal'] else "❌ Not causal"
    print(f"\n{status} (confidence: {result['confidence']:.3f})")
    print(f"   A: {ticket_a[:60]}...")
    print(f"   B: {ticket_b[:60]}...")

# 📋 Two-Stage Pipeline Integration

Use this causal classifier with your similarity model:

```python
# Stage 1: Find similar tickets using your similarity model
from sentence_transformers import SentenceTransformer

similarity_model = SentenceTransformer('models/v6_refactored_finetuned/...')
query_embedding = similarity_model.encode(new_ticket)

# Get top-K similar tickets
similarities = cosine_similarity([query_embedding], all_embeddings)[0]
top_k_indices = similarities.argsort()[-10:][::-1]

# Stage 2: Check for causal relationships
from sentence_transformers.cross_encoder import CrossEncoder

causal_model = CrossEncoder('models/causal_classifier/causal_YYYYMMDD_HHMM')

for idx in top_k_indices:
    candidate_ticket = tickets[idx]
    
    # Check both directions (A→B and B→A)
    score_ab = causal_model.predict([(new_ticket, candidate_ticket)])[0]
    score_ba = causal_model.predict([(candidate_ticket, new_ticket)])[0]
    
    if score_ab > 0.5:
        print(f"New ticket may have CAUSED ticket {idx}")
    elif score_ba > 0.5:
        print(f"Ticket {idx} may have CAUSED new ticket")
    else:
        print(f"Ticket {idx} is similar but not causally related")
```

**Note**: CrossEncoders are slower than bi-encoders but more accurate for pair classification. Only use on your top-K candidates, not the entire database.